## Overview

- The main purpose of this notebook is to process the 1000 .wav samples found in the local Data directory, slice them into smaller audio clips, convert these audio clips into mel-spectrograms, and convert the mel-spectrograms into numpy arrays.
- Once the audio data is processed, it can be converted into a dataset. The data set is broken up into training data and testing data. 
- The training and testing data is then used to train our model, which is then saved in /backend/genre_categorization.

In [3]:
import tensorflow as tf
import numpy as np
import os
import librosa
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [4]:
DATA_DIRECTORY = '../Data'
label_encoder = LabelEncoder()

In [5]:
def get_audio_data(path_name):
    """Function to create paths/genre arrays to allow for
    future conversion of audio files to mel_spectrograms."""
    paths, genres = [], []
    for root, _, files in os.walk(path_name):
        for name in files:
            filename = os.path.join(root, name)
            genre = os.path.split(root)[-1]
            paths.append(filename)
            genres.append(genre)
    return paths, genres

In [6]:
def create_melspectrogram(audio_file, sample_rate):
    """ Function to create a mel_spectrogram from a received audio_file"""
    mel_spectrogram = librosa.feature.melspectrogram(
          y=audio_file,
          sr=sample_rate,
          n_fft=2048,
          hop_length=512,
          n_mels=128)
    # db = decibel units
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)
    return mel_spectrogram_db

In [7]:
def pad_audio(mel_db_spect, max_length):
    """ Pads mel_spectrogram files to ensure that they are homogeneous
    in shape. """
    if mel_db_spect.shape[1] < max_length:
        padding = max_length - mel_db_spect.shape[1]
        mel_db_spect = np.pad(mel_db_spect, pad_width=((0,0), (0, padding)), mode='constant')
    else:
        mel_db_spect = mel_db_spect[:, :max_length]
    return mel_db_spect

In [8]:
def split_songs(audio_paths, genres, max_length=78):
    """ Function that splits received songs into smaller audio clips and
    converts them into mel-spectrograms to be used for training/prediction.
    Adapted from code found at: 
    https://github.com/chittalpatel/Music-Genre-Classification-GTZAN/blob/master/Music%20Genre%20Classification/CNN_train(1).ipynb
    """
    split_spects_mel_db = []
    split_genres = []
    window = 0.06
    overlap = 0.3

    for path, genre in tqdm(zip(audio_paths, genres), total=len(audio_paths),desc='Processing Audio Files'):  
        audio, sample_rate = librosa.load(path)  
        audio_shape = audio.shape[0]
        chunk = int(audio_shape * window)
        offset = int(chunk*(1 - overlap))
        individual_split_song = []

        # create array of smaller audio clips
        for i in range(0, audio_shape - chunk + offset, offset):
            individual_split_song.append(audio[i:i+chunk])
        
        # convert small clips into mel_spectrograms
        for sample in individual_split_song:
            if sample.shape[0] != chunk:
                continue
            mel_spec = librosa.feature.melspectrogram(y=sample, sr=sample_rate, n_fft=2048, hop_length=512, n_mels=128)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
            
            # Pad outputs to ensure uniformity 
            mel_spec_db = pad_audio(mel_spec_db, max_length)
            
            split_spects_mel_db.append(mel_spec_db)          
            split_genres.append(genre)
    
    return split_spects_mel_db, split_genres

In [9]:
paths, gens = get_audio_data(DATA_DIRECTORY)

In [10]:
spects, gens2 = split_songs(paths, gens)

Processing Audio Files: 100%|██████████| 1000/1000 [01:49<00:00,  9.12it/s]


Determine size of training set and test sets

In [11]:
spects_len = len(spects)
train_size = spects_len * .90
test_size = spects_len - train_size
train_take = train_size / 50
test_take = test_size / 50

Create Dataset

In [12]:
g1 = np.array(gens2)
s1 = np.array(spects)
genres_encoded = label_encoder.fit_transform(g1)
dataset = tf.data.Dataset.from_tensor_slices((s1, genres_encoded))
dataset = dataset.shuffle(len(s1))

In [13]:
final_data = dataset.cache()
final_data = final_data.batch(50)
final_data = final_data.prefetch(25)
train = final_data.take(int(train_take))
test = final_data.skip(int(train_take)).take(int(test_take))

CNN Model

In [14]:
spectrogram_height = 128
spectrogram_width = 78
num_channels = 1  #should be 1 because it is a numpy array, not a color image.
num_classes = 10

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(spectrogram_height, spectrogram_width, num_channels)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') 
])

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Train Model

In [16]:
model.fit(train, epochs=15, validation_data=test)

Epoch 1/15
414/414 [==============================] - 101s 238ms/step - loss: 1.7050 - accuracy: 0.4501 - val_loss: 1.2181 - val_accuracy: 0.5730
Epoch 2/15
414/414 [==============================] - 74s 180ms/step - loss: 1.0748 - accuracy: 0.6238 - val_loss: 0.9186 - val_accuracy: 0.6874
Epoch 3/15
414/414 [==============================] - 74s 178ms/step - loss: 0.9157 - accuracy: 0.6797 - val_loss: 0.9054 - val_accuracy: 0.6765
Epoch 4/15
414/414 [==============================] - 72s 174ms/step - loss: 0.7959 - accuracy: 0.7273 - val_loss: 0.8741 - val_accuracy: 0.6943
Epoch 5/15
414/414 [==============================] - 72s 174ms/step - loss: 0.7032 - accuracy: 0.7602 - val_loss: 0.9708 - val_accuracy: 0.6657
Epoch 6/15
414/414 [==============================] - 74s 178ms/step - loss: 0.6342 - accuracy: 0.7838 - val_loss: 0.8060 - val_accuracy: 0.7252
Epoch 7/15
414/414 [==============================] - 76s 185ms/step - loss: 0.5598 - accuracy: 0.8052 - val_loss: 0.7430 - val_a

Save Model

In [17]:
model.save('../genre_categorization')

INFO:tensorflow:Assets written to: ../genre_categorization\assets


INFO:tensorflow:Assets written to: ../genre_categorization\assets
